## Width-Scale Bar Chart

This notebook contains a possible implementation, using [Altair](https://altair-viz.github.io/) and [pandas](https://pandas.pydata.org/), of the Width-Scale Bar Chart (Höhn et al., 2020), an alternative to the typical bar chart to visualize data with a wide value/bar height range.

You can also check the rendered notebook [here](https://nbviewer.jupyter.org/github/joaopalmeiro/datavis-python-playground/blob/master/altair-width-scale-bar-chart/width-scale-bar-chart.ipynb).

**References**:

- Höhn, M., Wunderlich, M., Ballweg, K. & Landesberger, T. v. (2020). Width-Scale Bar Charts for Data with Large Value Range (A. Kerren, C. Garth & G. E. Marai, Eds.). In A. Kerren, C. Garth & G. E. Marai (Eds.), *Eurovis 2020 - short papers*, The Eurographics Association. https\://doi.org/10.2312/evs.20201056 [🔗](https://diglib.eg.org/handle/10.2312/evs20201056)

In [1]:
import pandas as pd
import altair as alt
from typing import Dict

In [2]:
alt.__version__

'4.1.0'

In [3]:
COLORS = {"white": "#FFFFFF", "light_gray": "#EBEBEB", "black": "#44475A"}


def custom_theme_tooltip():
    from IPython.core.display import HTML

    # More info: https://github.com/vega/vega-tooltip/blob/master/vega-tooltip.scss
    return display(
        HTML(
            f"""
            <style>
                #vg-tooltip-element.vg-tooltip.custom-theme {{
                    color: {COLORS["black"]};
                    border: 1px solid {COLORS["light_gray"]};
                    font-family: Roboto;
                    font-size: 11px;
                }}

                #vg-tooltip-element.vg-tooltip.custom-theme td.key {{
                    color: {COLORS["black"]};
                    font-weight: bold;
                }}
            </style>
            """
        )
    )


def custom_theme() -> Dict[str, Dict[str, object]]:
    font = "Roboto"

    return {
        "config": {
            "title": {"font": font, "color": COLORS["black"]},
            "axisX": {
                "labelFont": font,
                "titleFont": font,
                "gridColor": COLORS["light_gray"],
                "labelColor": COLORS["black"],
                "tickColor": COLORS["black"],
                "titleColor": COLORS["black"],
                "domainColor": COLORS["black"],
            },
            "axisY": {
                "labelFont": font,
                "titleFont": font,
                "gridColor": COLORS["light_gray"],
                "labelColor": COLORS["black"],
                "tickColor": COLORS["black"],
                "titleColor": COLORS["black"],
                "domainColor": COLORS["black"],
                "titleAngle": 0,
                "titleAlign": "left",
                "titleY": -5,
                "titleX": 0,
            },
            "header": {
                "labelFont": font,
                "titleFont": font,
                "labelColor": COLORS["black"],
                "titleColor": COLORS["black"],
            },
            "legend": {
                "labelFont": font,
                "titleFont": font,
                "labelColor": COLORS["black"],
                "titleColor": COLORS["black"],
            },
            "text": {"color": COLORS["black"]},
            "background": COLORS["white"],
            "view": {"fill": COLORS["white"]},
        }
    }


def set_alt_tooltip_theme(tooltip_theme_name: str) -> str:
    if tooltip_theme_name in ["light", "dark"]:
        return tooltip_theme_name
    else:
        custom_theme_tooltip()
        return tooltip_theme_name


THEMES = {"custom": custom_theme}


def set_alt_aesthetic(
    theme_name: str = "custom", tooltip_theme_name: str = "custom"
) -> None:
    tooltip_theme = set_alt_tooltip_theme(tooltip_theme_name)

    # More info: https://github.com/vega/vega-embed
    alt.renderers.enable(
        "default",
        embed_options={
            "actions": {
                "export": True,
                "source": False,
                "compiled": False,
                "editor": True,
            },
            "scaleFactor": 5,
            "i18n": {"PNG_ACTION": "Save as PNG", "SVG_ACTION": "Save as SVG"},
            "tooltip": {"theme": tooltip_theme},
            "renderer": "svg",
        },
    )

    alt.themes.register(theme_name, THEMES.get(theme_name))
    alt.themes.enable(theme_name)


set_alt_aesthetic()

In [4]:
data = dict(
    category=["A", "B", "C", "D", "E", "F", "G", "H"], 
    value=[800, 100, 4, 20, 400, 2000, 10, 60] 
)

In [5]:
def numeric_to_mantissa_and_exponent(value):
    exponent = int(len(str(abs(value)).split(".")[0]) - 1)
    mantissa = float(value * 10 ** -exponent)
    multiplier = int(10 ** exponent)

    return mantissa, exponent, multiplier


def end_x(value):
    return 10 * (value + 1)

In [6]:
def create_dataset(data):
    df = pd.DataFrame(data)
    df["mantissa"], df["exponent"], df["multiplier"] = zip(
        *df["value"].map(numeric_to_mantissa_and_exponent)
    )
    df["end"] = df["exponent"].map(end_x)
    df["end"] = df["end"].cumsum()
    df["start"] = df["end"].shift(fill_value=0)
    df["middle"] = ((df["end"] - df["start"]) / 2) + df["start"]
    df["original"] = (
        df["value"].map(str)
        + " = "
        + df["mantissa"].map("{0:g}".format)
        + " × 10^"
        + df["exponent"].map(str)
    )

    return df

In [7]:
df = create_dataset(data)

In [8]:
df.head()

,category,value,mantissa,exponent,multiplier,end,start,middle,original
0,A,800,8.0,2,100,30,0,15.0,800 = 8 × 10^2
1,B,100,1.0,2,100,60,30,45.0,100 = 1 × 10^2
2,C,4,4.0,0,1,70,60,65.0,4 = 4 × 10^0
3,D,20,2.0,1,10,90,70,80.0,20 = 2 × 10^1
4,E,400,4.0,2,100,120,90,105.0,400 = 4 × 10^2


In [9]:
def wsb_chart(data, w=600, h=300):
    base = alt.Chart(data, width=w, height=h)

    bar = base.mark_rect(xOffset=1.0, x2Offset=0.5).encode(
        x=alt.X(
            "start:Q",
            axis=alt.Axis(
                titleY=(-0.5 + 22),
                labels=False,
                title="Category",
                grid=False,
                values=data["middle"].to_list(),
            ),
        ),
        x2=alt.X2("end:Q"),
        y=alt.Y(
            "mantissa:Q",
            axis=alt.Axis(title="Mantissa"),
            scale=alt.Scale(domain=[0, 10]),
        ),
        color=alt.Color(
            "multiplier:O",
            title="Magnitude Multiplier",
            legend=alt.Legend(labelExpr="'× ' + datum.value"),
            scale=alt.Scale(scheme="reds"),
        ),
        tooltip=[
            alt.Tooltip("category", title="Category"),
            alt.Tooltip("original", title="Original Value"),
            alt.Tooltip("mantissa", title="Mantissa"),
            alt.Tooltip("multiplier", title="Magnitude Multiplier"),
        ],
    )

    # Altair/Vega-Lite:
    # default `fontSize` = 11
    # default `tickSize` = 5
    # default `labelPadding` = 2
    # default `translate` = 0.5

    text = base.mark_text(
        align="center", dx=0.5, baseline="middle", fontSize=11
    ).encode(
        x=alt.X("middle:Q"),
        y=alt.value(h + (11 / 2) + 5 + 2 + 0.5),
        text=alt.Text("category:N"),
    )

    return (bar + text).configure_view(strokeWidth=0)

In [10]:
wsb_chart(df)

alt.LayerChart(...)